In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import string
import spacy
from spacy.lang.de.stop_words import STOP_WORDS
import pandas as pd
from tqdm import tqdm

In [2]:
sc = SparkContext('local')
spark = SparkSession(sc)

In [3]:
# Load parquet file into spark dataframe
parquetFile = spark.read.parquet(
    "../data/customer_feedbacks/part-00000-985ad763-a6d6-4ead-a6dd-c02279e9eeba-c000.snappy.parquet")
parquetFile.createOrReplaceTempView("parquetFile")
parquetFile2 = spark.read.parquet(
    "../data/customer_feedbacks_cat/part-00000-4820af87-4b19-4958-a7a6-7ed03b76f1b1-c000.snappy.parquet")
parquetFile2.createOrReplaceTempView("parquetFile2")

In [4]:
# Select columns which are needed
df = spark.sql("""
SELECT
    /* T0.KATEGORIE_2     AS CATEGORY_2,
    T0.KATEGORIE_1     AS CATEGORY_1,
    T1.ERGEBNISSATZ_ID AS RESPONSE_ID,
    T0.STIMMUNG          AS SENTIMENT,
    T1.DATUM_ID        AS DATE, */
    T1.ANTWORT_WERT    AS TEXT
FROM
    parquetFile2 T0,
    parquetFile T1
WHERE
    T0.KATEGORIE_1_ID = T1.KATEGORIE_1_ID
    AND T0.KATEGORIE_2_ID = T1.KATEGORIE_2_ID
    AND T0.STIMMUNG_ID = T1.STIMMUNG_ID
    AND (NOT T1.ANTWORT_WERT IS NULL
        AND (T1.UMFRAGE_KATEGORIE_ID = 1
            AND (T1.GRUPPE_ID = 170
                OR T1.GRUPPE_ID = 171)))
""")

In [5]:
df.show()

+--------------------+
|                TEXT|
+--------------------+
|Habe bis heute me...|
|Habe bis heute me...|
|  Bin stinke sauer! |
|Teuer, aber gut. ...|
|Teuer, aber gut. ...|
|Bisher war ich mi...|
|Bisher war ich mi...|
|Bisher damit sehr...|
|                   -|
|Ich habe gute Erf...|
|da es super gewor...|
|gute Qualität, sc...|
|gute Qualität, sc...|
|Es ist einfach mi...|
|Es ist einfach mi...|
|Ich habe vor läng...|
|Ich habe vor läng...|
|Schnelle Bearbeit...|
|Schnelle Bearbeit...|
|wir waren super z...|
+--------------------+
only showing top 20 rows



In [11]:
%%time
# Convert Spark dataframe to Pandas dataframe
df_pd = df.dropDuplicates().toPandas()[:0]

Wall time: 27.7 s


In [15]:
floor(-0.1)

NameError: name 'floor' is not defined

In [27]:
# Prepare data for Gensim LDA
stopwords = list(STOP_WORDS)
nlp = spacy.load('de')
def text_preproc(sentence):
    mytokens = nlp.tokenizer(sentence)
    mytokens = [word.lemma_.lower().strip() for word in mytokens]
    mytokens = [word for word in mytokens if word not in stopwords and word not in string.punctuation]
    return mytokens

In [28]:
tqdm.pandas()
data = df_pd['TEXT'].progress_apply(text_preproc)





  0%|                                                                                    | 0/896683 [00:00<?, ?it/s]



  0%|                                                                        | 116/896683 [00:00<12:53, 1159.48it/s]



  0%|                                                                        | 300/896683 [00:00<11:28, 1301.41it/s]



  0%|                                                                        | 490/896683 [00:00<10:24, 1434.04it/s]



  0%|                                                                        | 690/896683 [00:00<09:31, 1567.03it/s]



  0%|                                                                        | 924/896683 [00:00<08:36, 1735.70it/s]



  0%|                                                                       | 1138/896683 [00:00<08:06, 1839.79it/s]



  0%|                                                                       | 1375/896683 [00:00<07:33, 1972.14it/s]



  0%|▏                              

  4%|██▉                                                                   | 37723/896683 [00:14<05:14, 2734.27it/s]



  4%|██▉                                                                   | 37998/896683 [00:14<05:17, 2706.63it/s]



  4%|██▉                                                                   | 38297/896683 [00:14<05:08, 2785.82it/s]



  4%|███                                                                   | 38584/896683 [00:14<05:06, 2802.69it/s]



  4%|███                                                                   | 38870/896683 [00:14<05:04, 2819.24it/s]



  4%|███                                                                   | 39178/896683 [00:14<04:57, 2884.63it/s]



  4%|███                                                                   | 39468/896683 [00:14<05:01, 2846.69it/s]



  4%|███                                                                   | 39785/896683 [00:14<04:52, 2928.49it/s]



  4%|███▏                               

  9%|██████                                                                | 77014/896683 [00:28<05:14, 2605.15it/s]



  9%|██████                                                                | 77278/896683 [00:28<05:26, 2510.76it/s]



  9%|██████                                                                | 77532/896683 [00:28<05:31, 2468.06it/s]



  9%|██████                                                                | 77814/896683 [00:28<05:19, 2564.05it/s]



  9%|██████                                                                | 78073/896683 [00:28<05:22, 2541.46it/s]



  9%|██████                                                                | 78331/896683 [00:28<05:21, 2545.34it/s]



  9%|██████▏                                                               | 78587/896683 [00:28<05:23, 2527.13it/s]



  9%|██████▏                                                               | 78848/896683 [00:28<05:20, 2551.38it/s]



  9%|██████▏                            

 13%|████████▊                                                            | 114396/896683 [00:42<04:58, 2623.50it/s]



 13%|████████▊                                                            | 114690/896683 [00:42<04:49, 2703.88it/s]



 13%|████████▊                                                            | 114962/896683 [00:42<04:50, 2692.30it/s]



 13%|████████▊                                                            | 115243/896683 [00:42<04:46, 2726.56it/s]



 13%|████████▉                                                            | 115518/896683 [00:42<04:45, 2733.55it/s]



 13%|████████▉                                                            | 115818/896683 [00:42<04:38, 2800.51it/s]



 13%|████████▉                                                            | 116135/896683 [00:42<04:29, 2894.12it/s]



 13%|████████▉                                                            | 116426/896683 [00:42<04:29, 2898.79it/s]



 13%|████████▉                          

 17%|███████████▋                                                         | 152590/896683 [00:56<04:20, 2855.99it/s]



 17%|███████████▊                                                         | 152878/896683 [00:56<04:28, 2772.19it/s]



 17%|███████████▊                                                         | 153157/896683 [00:56<04:47, 2584.52it/s]



 17%|███████████▊                                                         | 153420/896683 [00:56<04:50, 2560.08it/s]



 17%|███████████▊                                                         | 153717/896683 [00:56<04:38, 2663.49it/s]



 17%|███████████▊                                                         | 153987/896683 [00:56<04:40, 2643.25it/s]



 17%|███████████▊                                                         | 154271/896683 [00:56<04:35, 2699.04it/s]



 17%|███████████▉                                                         | 154543/896683 [00:57<04:48, 2574.68it/s]



 17%|███████████▉                       

...

 82%|████████████████████████████████████████████████████████▌            | 735106/896683 [04:43<01:23, 1946.48it/s]



 82%|████████████████████████████████████████████████████████▌            | 735344/896683 [04:43<01:18, 2058.98it/s]



 82%|████████████████████████████████████████████████████████▌            | 735579/896683 [04:43<01:15, 2138.42it/s]



 82%|████████████████████████████████████████████████████████▌            | 735814/896683 [04:43<01:13, 2191.65it/s]



 82%|████████████████████████████████████████████████████████▋            | 736065/896683 [04:43<01:10, 2272.20it/s]



 82%|████████████████████████████████████████████████████████▋            | 736303/896683 [04:44<01:09, 2296.84it/s]



 82%|████████████████████████████████████████████████████████▋            | 736580/896683 [04:44<01:06, 2420.85it/s]



 82%|████████████████████████████████████████████████████████▋            | 736843/896683 [04:44<01:04, 2473.31it/s]



 82%|███████████████████████████████████

 86%|███████████████████████████████████████████████████████████▍         | 773207/896683 [04:57<00:49, 2478.83it/s]



 86%|███████████████████████████████████████████████████████████▌         | 773478/896683 [04:57<00:48, 2536.83it/s]



 86%|███████████████████████████████████████████████████████████▌         | 773759/896683 [04:57<00:47, 2606.04it/s]



 86%|███████████████████████████████████████████████████████████▌         | 774030/896683 [04:58<00:46, 2636.14it/s]



 86%|███████████████████████████████████████████████████████████▌         | 774296/896683 [04:58<00:46, 2643.24it/s]



 86%|███████████████████████████████████████████████████████████▌         | 774574/896683 [04:58<00:45, 2682.79it/s]



 86%|███████████████████████████████████████████████████████████▌         | 774844/896683 [04:58<00:48, 2516.41it/s]



 86%|███████████████████████████████████████████████████████████▋         | 775099/896683 [04:58<00:49, 2474.89it/s]



 86%|███████████████████████████████████

 90%|██████████████████████████████████████████████████████████████▍      | 810953/896683 [05:12<00:29, 2932.57it/s]



 90%|██████████████████████████████████████████████████████████████▍      | 811267/896683 [05:12<00:28, 2991.85it/s]



 91%|██████████████████████████████████████████████████████████████▍      | 811567/896683 [05:12<00:29, 2915.71it/s]



 91%|██████████████████████████████████████████████████████████████▍      | 811860/896683 [05:12<00:30, 2763.02it/s]



 91%|██████████████████████████████████████████████████████████████▍      | 812139/896683 [05:12<00:31, 2701.82it/s]



 91%|██████████████████████████████████████████████████████████████▌      | 812414/896683 [05:12<00:31, 2708.10it/s]



 91%|██████████████████████████████████████████████████████████████▌      | 812698/896683 [05:12<00:30, 2738.50it/s]



 91%|██████████████████████████████████████████████████████████████▌      | 812974/896683 [05:12<00:30, 2744.91it/s]



 91%|███████████████████████████████████

 94%|█████████████████████████████████████████████████████████████████    | 845146/896683 [05:26<00:20, 2538.11it/s]



 94%|█████████████████████████████████████████████████████████████████    | 845444/896683 [05:26<00:19, 2656.24it/s]



 94%|█████████████████████████████████████████████████████████████████    | 845712/896683 [05:26<00:19, 2601.61it/s]



 94%|█████████████████████████████████████████████████████████████████    | 845987/896683 [05:26<00:19, 2644.09it/s]



 94%|█████████████████████████████████████████████████████████████████    | 846266/896683 [05:26<00:18, 2678.53it/s]



 94%|█████████████████████████████████████████████████████████████████▏   | 846555/896683 [05:26<00:18, 2730.81it/s]



 94%|█████████████████████████████████████████████████████████████████▏   | 846838/896683 [05:26<00:18, 2751.89it/s]



 94%|█████████████████████████████████████████████████████████████████▏   | 847114/896683 [05:26<00:18, 2737.86it/s]



 95%|███████████████████████████████████

 98%|███████████████████████████████████████████████████████████████████▊ | 880866/896683 [05:40<00:07, 2127.89it/s]



 98%|███████████████████████████████████████████████████████████████████▊ | 881085/896683 [05:40<00:07, 2084.88it/s]



 98%|███████████████████████████████████████████████████████████████████▊ | 881321/896683 [05:41<00:07, 2160.20it/s]



 98%|███████████████████████████████████████████████████████████████████▊ | 881567/896683 [05:41<00:06, 2236.11it/s]



 98%|███████████████████████████████████████████████████████████████████▊ | 881823/896683 [05:41<00:06, 2324.00it/s]



 98%|███████████████████████████████████████████████████████████████████▉ | 882091/896683 [05:41<00:06, 2413.93it/s]



 98%|███████████████████████████████████████████████████████████████████▉ | 882359/896683 [05:41<00:05, 2487.97it/s]



 98%|███████████████████████████████████████████████████████████████████▉ | 882650/896683 [05:41<00:05, 2594.26it/s]



 98%|███████████████████████████████████